# Data Collection

## 1.1 Import Libraries


In [ ]:
# Import necessary libraries
import os
import sys
import joblib
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import dump
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Append the correct path for custom module imports
sys.path.append(os.path.abspath('../src'))

# Import custom modules
from preprocess import Preprocessor
from model import ModelTrainer


## 1.2 Import Warnings and Logging Libraries


In [ ]:
# Warnings and logging
import warnings
import logging
warnings.filterwarnings("ignore")

# Append the correct path for custom module imports
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../data'))
sys.path.append(os.path.abspath('../notebook/models'))

# Optional custom logging (currently commented out)
# from custom_logging import info_logger, error_logger


## 1.3 Commented Out Imports for Future Reference


In [ ]:
# from data_processing import load_data, clean_data, handle_missing_values
# from sale_analysis import (
#     plot_sales_distribution,
#     compare_sales_holidays,
#     seasonal_behavior,
#     correlation_analysis,
#     promo_effect,
#     effective_promo_deployment,
#     customer_behavior_trends,
#     weekday_openings,
#     assortment_type_impact,
#     competitor_distance_impact,
#     new_competitor_effects, 
#     plot_promo_distribution,
#     plot_sales_during_holidays,
#     plot_sales_customers_corr, 
#     plot_store_corr
# )


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## 1.2 Load the Data


In [ ]:
# Define the paths to the datasets
train_path = '/kaggle/input/rossmann-store-sales/train.csv'
test_path = '/kaggle/input/rossmann-store-sales/test.csv'
store_path = '/kaggle/input/rossmann-store-sales/store.csv'

# Initialize the Preprocessor class
preprocessor = Preprocessor(train_path, test_path, store_path)

# Load the datasets
train_data, test_data, store_data = preprocessor.load_data()


# Data Preprocessing

## 2.1 Summary of Dataset


In [ ]:
# Summarize the dataset
preprocessor.summarize_data()


## 2.2 Descriptive Statistics for Training Data


In [ ]:
# Display descriptive statistics for the training dataset
train_data.describe()


## 2.3 Data Types and Descriptive Statistics for Test and Store Data


In [ ]:
# Display data types for the test dataset
print(test_data.dtypes)

# Display data types for the store dataset
print(store_data.dtypes)

# Display descriptive statistics for the store dataset
store_data.describe()


## 2.4 Data Cleaning

### 2.4.1 Checking Missing Values in Training Data


In [ ]:
# Check for missing values in the training dataset
train_data.isna().sum()


### 2.4.2 Checking Missing Values in Testing Data


In [ ]:
# Check for missing values in the testing dataset
test_data.isna().sum()


### 2.4.3 Checking Missing Values in Store Data


In [ ]:
# Check for missing values in the store dataset
store_data.isnull().sum()


## 2.5 Merge the Dataset


In [ ]:
# Merge the datasets using the Preprocessor class
merged_data = preprocessor.merge_datasets()

# Display the head of the final combined dataset
merged_data.head()


In [ ]:
# Check the shape of the merged dataset
merged_data.shape


### 2.5.2 Checking Missing Values in the Merged Dataset


In [ ]:
# Check for missing values in the merged dataset
merged_data.isnull().sum()


## 2.6 Handling Missing Values


In [ ]:
# Handle missing values in the merged dataset
cleaned_data = preprocessor.handle_missing_values()

# Verify if there are still missing values
print(cleaned_data.isnull().sum())


## 2.7 Feature Engineering




In [ ]:
# Perform feature engineering on the dataset
X, y = preprocessor.feature_engineering()

# Display the head of the engineered dataset
X.head()


In [ ]:
# The feature engineering step already includes extracting date-based features
# This step is handled within the Preprocessor class

# Perform feature engineering on the dataset (already includes extracting date-based features)
X, y = preprocessor.feature_engineering()

# Display the head of the engineered dataset
X.head()


In [ ]:
# The feature engineering step already includes handling missing values and label encoding
# Additional holiday features can be implemented if a holiday dataset is available

# Perform feature engineering on the dataset (already includes label encoding and handling missing values)
X, y = preprocessor.feature_engineering()

# Display the head of the engineered dataset
X.head()


In [ ]:
# The feature engineering step already includes scaling the data
# This step is handled within the Preprocessor class

# Perform feature engineering on the dataset (already includes scaling the data)
X, y = preprocessor.feature_engineering()

# Display the head of the scaled dataset
X.head()


In [ ]:
# Define the target (sales) and features using the Preprocessor class
X, y = preprocessor.feature_engineering()

# Display the head of the features and target
X.head(), y.head()


In [ ]:
# Check for non-numeric columns in the feature set
non_numeric_columns = X.select_dtypes(include=['object']).columns
print(f"Non-numeric columns: {non_numeric_columns}")

# Apply One-Hot Encoding to categorical columns (if any)
X_encoded = pd.get_dummies(X, columns=non_numeric_columns)

# Split data into train and test sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


## 2.10 Training the XGBoost Model


In [ ]:
# Initialize the ModelTrainer class with features and target
model_trainer = ModelTrainer(X_train, y_train)

# Train the XGBoost model within a pipeline
xgb_pipeline = model_trainer.train_xgboost()

# Predict and evaluate on training data
y_pred_train_xgb = xgb_pipeline.predict(X_train)

# Predict and evaluate on testing data
y_pred_test_xgb = xgb_pipeline.predict(X_test)


## 2.11 Model Evaluation


In [ ]:
# Define the evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, mae, r2

# Evaluate the model on training data
train_mse, train_mae, train_r2 = evaluate_model(y_train, y_pred_train_xgb)
print(f"Training Data - MSE: {train_mse}, MAE: {train_mae}, R2: {train_r2}")

# Evaluate the model on testing data
test_mse, test_mae, test_r2 = evaluate_model(y_test, y_pred_test_xgb)
print(f"Testing Data - MSE: {test_mse}, MAE: {test_mae}, R2: {test_r2}")


In [ ]:
# Evaluate the XGBoost model on the training set
mse_train_xgb, mae_train_xgb, r2_train_xgb = evaluate_model(y_train, y_pred_train_xgb)
print(f"XGBoost Training Set - MSE: {mse_train_xgb}, MAE: {mae_train_xgb}, R2: {r2_train_xgb}")

# Evaluate the XGBoost model on the test set
mse_test_xgb, mae_test_xgb, r2_test_xgb = evaluate_model(y_test, y_pred_test_xgb)
print(f"XGBoost Test Set - MSE: {mse_test_xgb}, MAE: {mae_test_xgb}, R2: {r2_test_xgb}")


## 2.13 Training the Random Forest Model


In [ ]:
# Initialize the ModelTrainer class with features and target
model_trainer = ModelTrainer(X_train, y_train)

# Train the Random Forest model within a pipeline
rf_pipeline = model_trainer.train_random_forest()

# Predict and evaluate on training data
y_pred_train_rf = rf_pipeline.predict(X_train)

# Predict and evaluate on testing data
y_pred_test_rf = rf_pipeline.predict(X_test)


## 2.14 Choosing a Loss Function and Evaluating the Random Forest Model


In [ ]:
# Define evaluation metrics
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, mae, r2

# Predict and evaluate on training data
y_pred_train_rf = rf_pipeline.predict(X_train)

# Predict and evaluate on testing data
y_pred_test_rf = rf_pipeline.predict(X_test)

# Evaluate the model on training set
mse_train_rf, mae_train_rf, r2_train_rf = evaluate_model(y_train, y_pred_train_rf)
print(f"Random Forest Training Set - MSE: {mse_train_rf}, MAE: {mae_train_rf}, R2: {r2_train_rf}")

# Evaluate the model on test set
mse_test_rf, mae_test_rf, r2_test_rf = evaluate_model(y_test, y_pred_test_rf)
print(f"Random Forest Test Set - MSE: {mse_test_rf}, MAE: {mae_test_rf}, R2: {r2_test_rf}")


In [ ]:
# Extract the RandomForest model from the pipeline
rf_model = rf_pipeline.named_steps['model']

# Get feature importances from the RandomForest model
importances = rf_model.feature_importances_

# Retrieve feature names after encoding
feature_names = X_encoded.columns

# Check the length of feature names and feature importances
print(f"Length of feature names: {len(feature_names)}")
print(f"Length of feature importances: {len(importances)}")

# Display the feature importances with corresponding feature names
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)


In [ ]:
# Set the number of top features to display
top_n = 10

# Plotting the top n important features
plt.figure(figsize=(12, 6))
plt.barh(feature_importance_df['Feature'][:top_n], feature_importance_df['Importance'][:top_n], color='skyblue')
plt.xlabel('Importance')
plt.title('Top Important Features from RandomForest')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()


In [ ]:
# Estimate the confidence interval based on the standard deviation of predictions
y_pred_std = np.std([tree.predict(X_test) for tree in rf_model.estimators_], axis=0)
confidence_interval = 1.96 * y_pred_std  # 95% confidence interval

# Plotting the predictions with confidence intervals
plt.figure(figsize=(10, 6))
plt.errorbar(y_test.index, y_pred_test_rf, yerr=confidence_interval, fmt='o', ecolor='r', capthick=2, label="Confidence Interval")
plt.scatter(y_test.index, y_test, color='blue', label='Actual Values', alpha=0.5)
plt.title("Predictions with 95% Confidence Interval")
plt.xlabel("Sample Index")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()


## 2.18 Saving the Model with Timestamp


In [ ]:
# Import necessary libraries
import os
import joblib
from datetime import datetime

# Define the models directory within the Kaggle environment
models_dir = '../notebook/models'  # Use relative path for Kaggle

# Ensure the directory exists
os.makedirs(models_dir, exist_ok=True)

# Serialize model with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_filename = f"{models_dir}/sales_model_{timestamp}.pkl"
joblib.dump(pipeline, model_filename)

print(f"Model saved as {model_filename}")


In [ ]:
# Import necessary libraries
import os
import sys
import joblib
from datetime import datetime

# Correct the path
sys.path.append(os.path.abspath('../notebook/models'))

# Ensure the directory exists
os.makedirs('../notebook/models', exist_ok=True)

# Serialize model with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_filename = f"../notebook/models/sales_model_{timestamp}.pkl"
joblib.dump(pipeline, model_filename)

print(f"Model saved as {model_filename}")


In [ ]:
# Prepare time series data
time_steps = 60

def create_lagged_data(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

sales = df['Sales'].values
scaled_sales = (sales - sales.mean()) / sales.std()  # Scale to (-1, 1)

# Create supervised learning data for LSTM
X_lstm, y_lstm = create_lagged_data(scaled_sales, time_steps)

# Split into train/test sets
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)),
    LSTM(50),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Reshape input for LSTM
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], X_train_lstm.shape[1], 1))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], X_test_lstm.shape[1], 1))

# Train the model
history = model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=64, validation_data=(X_test_lstm, y_test_lstm))

# Predict with LSTM
y_pred_lstm = model.predict(X_test_lstm)
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)
print(f"LSTM Model - Test Set MSE: {mse_lstm}")


## 2.21 Calculating Predictions and Metrics for the LSTM Model


In [ ]:
# Calculate predictions
y_pred_lstm = model.predict(X_test_lstm)

# Rescale predictions back to original values
y_pred_lstm_rescaled = y_pred_lstm * sales.std() + sales.mean()
y_test_lstm_rescaled = y_test_lstm * sales.std() + sales.mean()

# Calculate metrics
mae_lstm = mean_absolute_error(y_test_lstm_rescaled, y_pred_lstm_rescaled)
mse_lstm = mean_squared_error(y_test_lstm_rescaled, y_pred_lstm_rescaled)
r2_lstm = r2_score(y_test_lstm_rescaled, y_pred_lstm_rescaled)

print(f"LSTM Model - Test Set MAE: {mae_lstm:.2f}")
print(f"LSTM Model - Test Set MSE: {mse_lstm:.2f}")
print(f"LSTM Model - Test Set R²: {r2_lstm:.2f}")


## 2.22 Plotting LSTM Model Predictions vs Actual Sales


In [ ]:
# Plotting the results
plt.figure(figsize=(14, 7))
plt.plot(y_test_lstm_rescaled, label='Actual Sales', color='blue')
plt.plot(y_pred_lstm_rescaled, label='Predicted Sales', color='orange')
plt.title('LSTM Model Predictions vs Actual Sales')
plt.xlabel('Samples')
plt.ylabel('Sales')
plt.legend()
plt.show()


In [ ]:
# Import necessary libraries
import joblib
from datetime import datetime

# Define a filename with a timestamp
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
lstm_model_filename = f"../notebook/models/lstm_model_{timestamp}.h5"

# Save the model
model.save(lstm_model_filename)
print(f'LSTM model serialized to {lstm_model_filename}')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Scaling the sales data
sales = df['Sales'].values
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_sales = scaler.fit_transform(sales.reshape(-1, 1))

# Create lagged data for LSTM
def create_lagged_data(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 60
X_lstm, y_lstm = create_lagged_data(scaled_sales, time_steps)

# Split data into train and test sets
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# Reshape input to be 3D [samples, time steps, features]
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], X_train_lstm.shape[1], 1))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], X_test_lstm.shape[1], 1))

# Build enhanced LSTM model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)),
    Dropout(0.2),  # Add dropout for regularization
    LSTM(100, return_sequences=True),  # Additional LSTM layer
    Dropout(0.2),  # Another dropout
    LSTM(50, return_sequences=False),  # Final LSTM layer
    Dense(50, activation='relu'),  # Dense layer to learn more complex representations
    Dense(1)  # Output layer for regression
])

# Compile the model with mean squared error loss
model.compile(optimizer='adam', loss='mse')

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=64, validation_data=(X_test_lstm, y_test_lstm), 
                    callbacks=[early_stop], verbose=2)

## 2.24 Predicting and Evaluating the Enhanced LSTM Model


In [ ]:
# Predict on test data
y_pred_lstm = model.predict(X_test_lstm)

# Rescale predictions back to original scale
y_pred_lstm_rescaled = scaler.inverse_transform(y_pred_lstm)
y_test_lstm_rescaled = scaler.inverse_transform(y_test_lstm)

# Evaluate model
mae_lstm = mean_absolute_error(y_test_lstm_rescaled, y_pred_lstm_rescaled)
mse_lstm = mean_squared_error(y_test_lstm_rescaled, y_pred_lstm_rescaled)
r2_lstm = r2_score(y_test_lstm_rescaled, y_pred_lstm_rescaled)

print(f"LSTM Model - Test Set MAE: {mae_lstm:.2f}")
print(f"LSTM Model - Test Set MSE: {mse_lstm:.2f}")
print(f"LSTM Model - Test Set R²: {r2_lstm:.2f}")

# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(y_test_lstm_rescaled, label='Actual Sales', color='blue')
plt.plot(y_pred_lstm_rescaled, label='Predicted Sales', color='orange')
plt.title('Enhanced LSTM Model Predictions vs Actual Sales')
plt.xlabel('Samples')
plt.ylabel('Sales')
plt.legend()
plt.show()


## 2.25 Saving the Model within the Kaggle Environment


In [ ]:
# Define the models directory within the Kaggle environment
models_dir = './models'  # Use relative path for Kaggle

# Ensure the directory exists
os.makedirs(models_dir, exist_ok=True)

# Serialize model with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_filename = f"{models_dir}/sales_model_{timestamp}.pkl"
joblib.dump(pipeline, model_filename)

print(f"Model saved as {model_filename}")
